<a href="https://colab.research.google.com/github/SupreethRao99/DeepCAPTCHA/blob/main/DeepCAPTCHA_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeepCAPTCHA
DeepCAPTHA is a ResNet architecture based convolutional neural network (CNN) trained on the [Chars74K-Fonts](http://www.ee.surrey.ac.uk/CVSSP/demos/chars74k/#download) Dataset. It has been built as part of a larger project which attempts to defeat simple CAPTCHAs. The Network trained in this notebook achieves a training accuracy of 89.41% and a validation accuracy of 88.00%.

The [dataset](https://www.kaggle.com/supreethrao/chars74kdigitalenglishfont) used in this notebook can also be found on kaggle

In [ ]:
# importing the required libraries
from google.colab import drive
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import zipfile
import os
import random
from shutil import copyfile
import datetime

In [ ]:
!nvidia-smi #displays the GPU allocated by google colab

Sun Apr  4 16:39:26 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

The dataset is stored on google drive. The dataset is then loaded onto colab and unzipped. training and testing directories are created for each class present in the dataset.

In [ ]:
drive.mount('/content/drive') 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
zip_ref = zipfile.ZipFile("/content/drive/MyDrive/English.zip", 'r')
zip_ref.extractall("/tmp")
zip_ref.close()

In [ ]:
# Training and testing directories are created with directory name being the
# label of the class. eg:images of the letter 'A' will be in directory 'A'
# similarly images of letter 'a' will be in directory 'a'.

# Note that directory names are case-sensitive.
os.mkdir('/tmp/CAPTCHA')
os.mkdir('/tmp/CAPTCHA/testing')
os.mkdir('/tmp/CAPTCHA/training')

for i in range(0,62):
  try:
    if i>= 0 and i<10: # for numbers 0-9
      os.mkdir('/tmp/CAPTCHA/training/'+chr(i+48))
      os.mkdir('/tmp/CAPTCHA/testing/'+chr(i+48))

    if i>= 10 and i<36: # for alphabets A-Z
      os.mkdir('/tmp/CAPTCHA/training/'+chr(i-10+65))
      os.mkdir('/tmp/CAPTCHA/testing/'+chr(i-10+65))
      
    if i>=36 and i<62: # for alphabets a-z
      os.mkdir('/tmp/CAPTCHA/training/'+chr(i-36+97))
      os.mkdir('/tmp/CAPTCHA/testing/'+chr(i-36+97))
      
  except OSError:
    print('directory creation failed')
    pass

the `split_data` function splits the dataset into training and testing sets randomly. the size of the training and testing set is determined by the `SPLIT_SIZE` parameter

In [ ]:
from random import shuffle
import shutil

def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    all_images = os.listdir(SOURCE)
    shuffle(all_images)
    splitting_index = round(SPLIT_SIZE*len(all_images))
    train_images = all_images[:splitting_index]
    test_images = all_images[splitting_index:]

    for img in train_images:
        src = os.path.join(SOURCE, img)
        dst = os.path.join(TRAINING, img)
        if os.path.getsize(src) <= 0:
            print(img+" is zero length, so ignoring!!")
        else:
            shutil.copyfile(src, dst)

    for img in test_images:
        src = os.path.join(SOURCE, img)
        dst = os.path.join(TESTING, img)
        if os.path.getsize(src) <= 0:
            print(img+" is zero length, so ignoring!!")
        else:
            shutil.copyfile(src, dst)


In [ ]:
split_size = 0.90
for i in range(0,62):
  if i>=0 and i<10:
    split_data('/tmp/English/Fnt/'+chr(i+48),
               '/tmp/CAPTCHA/training/'+chr(i+48),
               '/tmp/CAPTCHA/testing/'+chr(i+48),
               split_size)
  if i>=10 and i<36:
    split_data('/tmp/English/Fnt/'+chr(i-10+65)+"-1",
               '/tmp/CAPTCHA/training/'+chr(i-10+65),
               '/tmp/CAPTCHA/testing/'+chr(i-10+65),
               split_size)
  if i>=36 and i<62:
    split_data('/tmp/English/Fnt/'+chr(i-36+97)+"-2",
               '/tmp/CAPTCHA/training/'+chr(i-36+97),
               '/tmp/CAPTCHA/testing/'+chr(i-36+97),
               split_size)

# Creation of Model



In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

Images are augmented by rescaling, rotating , shearing, zooming and flipping. This provides a cheap and very effective way to provide more data for the model to learn from.

In [ ]:
TRAINING_DIR = '/tmp/CAPTCHA/training'
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   rotation_range=30,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest',
            preprocessing_function = tf.image.rgb_to_grayscale)
train_generator = train_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size = (32,32),
    batch_size = 16,
    class_mode = 'categorical'
)

VALIDATION_DIR = '/tmp/CAPTCHA/testing'
validation_datagen = ImageDataGenerator(rescale=1./255,
            preprocessing_function = tf.image.rgb_to_grayscale)
validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    target_size = (32,32),
    batch_size = 16,
    class_mode = 'categorical'
)

Found 56668 images belonging to 62 classes.
Found 6324 images belonging to 62 classes.


## The ResNet Model
the model described below uses a custom model based on the [ResNet architecture](https://arxiv.org/pdf/1512.03385.pdf) 

In [ ]:
import keras
from functools import partial
DefaultConv2D = partial(keras.layers.Conv2D, kernel_size=3, strides=1,
                        padding="SAME", use_bias=False)

class ResidualUnit(keras.layers.Layer):
    def __init__(self, filters, strides=1, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        self.main_layers = [
            DefaultConv2D(filters, strides=strides),
            keras.layers.BatchNormalization(),
            self.activation,
            DefaultConv2D(filters),
            keras.layers.BatchNormalization()]
        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [
                DefaultConv2D(filters, kernel_size=1, strides=strides),
                keras.layers.BatchNormalization()]

    def get_config(self):
      cfg = super().get_config()
      return cfg  

    def call(self, inputs):
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        return self.activation(Z + skip_Z)

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(DefaultConv2D(64, kernel_size=4, strides=2,
                        input_shape=[32, 32, 3]))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.MaxPool2D(pool_size=3, strides=2, padding="SAME"))
prev_filters = 64
for filters in [64] * 2 + [128] * 2 + [256] * 2 :
    strides = 1 if filters == prev_filters else 2
    model.add(ResidualUnit(filters, strides=strides))
    prev_filters = filters
model.add(keras.layers.GlobalAvgPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(62, activation="softmax"))

model.compile(loss="categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 16, 16, 64)        3072      
_________________________________________________________________
batch_normalization (BatchNo (None, 16, 16, 64)        256       
_________________________________________________________________
activation (Activation)      (None, 16, 16, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 8, 8, 64)          0         
_________________________________________________________________
residual_unit (ResidualUnit) (None, 8, 8, 64)          74240     
_________________________________________________________________
residual_unit_1 (ResidualUni (None, 8, 8, 64)          74240     
_________________________________________________________________
residual_unit_2 (ResidualUni (None, 4, 4, 128)         2

In [ ]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

# performance scheduling is used to reduce learning rate
# to improve model training accuracy

learning_rate_reduction = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy', factor=0.5, patience=2,min_lr=0.00001,mode='auto')

In [ ]:
history = model.fit(train_generator, epochs = 50,
                    validation_data=validation_generator,
          callbacks=[tensorboard_callback, learning_rate_reduction])

Epoch 1/50
3542/3542 [==============================] - 198s 47ms/step - loss: 2.1433 - accuracy: 0.4257 - val_loss: 1.0550 - val_accuracy: 0.6482
Epoch 2/50
3542/3542 [==============================] - 165s 46ms/step - loss: 0.8481 - accuracy: 0.7205 - val_loss: 0.6754 - val_accuracy: 0.7679
Epoch 3/50
3542/3542 [==============================] - 166s 47ms/step - loss: 0.7120 - accuracy: 0.7584 - val_loss: 0.5683 - val_accuracy: 0.7887
Epoch 4/50
3542/3542 [==============================] - 164s 46ms/step - loss: 0.6451 - accuracy: 0.7755 - val_loss: 0.5259 - val_accuracy: 0.8033
Epoch 5/50
3542/3542 [==============================] - 165s 47ms/step - loss: 0.5815 - accuracy: 0.7926 - val_loss: 0.5272 - val_accuracy: 0.8066
Epoch 6/50
3542/3542 [==============================] - 164s 46ms/step - loss: 0.5452 - accuracy: 0.8020 - val_loss: 0.5391 - val_accuracy: 0.8025
Epoch 7/50
3542/3542 [==============================] - 165s 46ms/step - loss: 0.5218 - accuracy: 0.8086 - val_loss: 0

The model is saved in the `saved_model` format and downloaded for use as part of the larger project.

In [ ]:
model.save('CAPTCHA-Model')

INFO:tensorflow:Assets written to: CAPTCHA-Model/assets


INFO:tensorflow:Assets written to: CAPTCHA-Model/assets


In [ ]:
!zip -r /content/CAPTCHA.zip /content/CAPTCHA-Model

  adding: content/CAPTCHA-Model/ (stored 0%)
  adding: content/CAPTCHA-Model/variables/ (stored 0%)
  adding: content/CAPTCHA-Model/variables/variables.index (deflated 76%)
  adding: content/CAPTCHA-Model/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: content/CAPTCHA-Model/saved_model.pb (deflated 91%)
  adding: content/CAPTCHA-Model/assets/ (stored 0%)


In [ ]:
from google.colab import files
files.download("/content/CAPTCHA.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>